In [ ]:
import pandas as pd 
import numpy as np
import time 

df = pd.read_parquet('../scripts/df_merged.parquet', engine='pyarrow')

In [ ]:
df.shape

In [ ]:
unique_ids = df['objectId'].unique().tolist()

In [ ]:
condition = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

In [ ]:
df[df['source'] == 0]

In [ ]:
df[['source','dc_weight']].head()

In [ ]:
len(unique_ids)

In [ ]:
start_time = time.time()



m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1
Q = df[['dc_flux']].iloc[:2*(m+1)]#.values # j:j+m ?

d = {k: 0 for k in unique_ids}
#print(d)

count = 0

for k in unique_ids:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn
        n = len(T)
        zeros = np.zeros(n, dtype=float)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        f = T['dc_flux'].values
        w = T['dc_weight'].values
        
        for j in range(0,m+1): 
            #f = T['dc_flux'].values[j*2:] # * 2 !!! for r and g right ? 
            #w = T['dc_sigflux_weight'].values[j*2:]
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        #print(len(s), len(s_n1))
        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
            
        #alpha = s[::2] + s[1::2] 
        #print(s)
        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
    if (d[k] <= factor).all():
        print(k, ": ", d[k],"\n")
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
(d['ZTF17aaaeclk']<= factor)

In [ ]:
d['ZTF17aaaeclk']

In [ ]:
Q

In [ ]:
df[df['objectId']=='ZTF17aaaeclk']

In [ ]:
alpha

In [ ]:
1.32646186*np.array((0.000286,0.000636))

#

# with l>0

In [ ]:
m=1
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))


this part takes a lot of time ! 

In [ ]:
chunk_size = 2 * (m + 1)
# Calculate the number of chunks
num_chunks = len(df[['dc_flux']]) // chunk_size
# Split the DataFrame into chunks and store them in an array
Q = [df[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]


In [ ]:
if m>0: #for j in range(1, max(1, m+1)): this isn't faster
    for j in range(1, m+1):
        Q.extend([df[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])

Q = pd.concat(Q) # here we transform Q from list of dataframes to a single data frame  
# we will be able to distinct the values by using l variable s


In [ ]:
Number_l = (len(Q)) // chunk_size 

d   = {k: [0] * Number_l for k in unique_ids[0:10]}
alp = {k: [0] * Number_l for k in unique_ids[0:10]}
a = {k: [0] * Number_l for k in unique_ids[0:10]}

In [ ]:
start_time = time.time()

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['dc_flux', 'dc_weight']]
    n = len(T)
    f = T['dc_flux'].values
    w = T['dc_weight'].values
    
    zeros = np.zeros(n, dtype=float)

    for l in range(Number_l): # I think we will changee it to while 
        #print(l)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        d[k][l]=0
        
        for j in range(0,m+1): 
            h = np.tile(Q[2*l*(m+1) +j*2: 2*l*(m+1) + j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
        #alp[k][l] = s
        
        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[2*l*(m+1) + j*2: 2*l*(m+1) +j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            # 2*l*(m+1) this part is to take from Q by l 
            
            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] 

        d[k][l] = dd[::2] + dd[1::2]
        
        a[k][l] =  d[k][l] <= factor   
            
        if (d[k][l] <= factor).all():
            print(k, ": ", d[k][l],"\n")
            count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
Number_l, len(Q), len(df)

In [ ]:
len(d[k][1])

In [ ]:
d.keys()

In [ ]:
alp[k][1]

In [ ]:
d

 j'ai complété l'implémentation `(partie with l>0)`, mais pour 'Q', j'ai actuellement toutes les valeurs possibles pour 'Q' (c'est-à-dire toutes les combinaisons possibles de valeurs de flux)
 dans ce cas le code fait tous les calculs des distances en stockant les valeurs True ou False si d<= factor (variable 'a') . 
 
Cependant, cela rend le calcul trop complexe(avec les donnees completes il ne marche pas (kernel interrepted )). Donc j'essaie de passer à l'approche sélective, mais je rencontre des difficultés avec la logique de sélection, surtout lorsque 'm' est supérieur à zéro.

 ma compréhension du logique de sélection :

- Lorsque 'm' = 0, si la valeur de 'd(i)'  > facteur, je considère 'f(i)' comme la nouvelle valeur de 'Q' (en faites, je prend tous les valeurs f(i) ou d(i)> facteur ce qui réduit 'Q' à ces valeurs)
    
- Mais lorsque 'm' >0 alors que 'Q' contient deux valeurs ou plus (supposons 'i' et 'i+1'), la condition est : si à la fois 'd(i)' et 'd(i+1)' sont supérieurs au facteur, alors 'Q_next' est composé de 'f(i)' et 'f(i+1)' n'est ce pas ?
    
    

#### Mr.  : 
m étant donné. (possiblement m>0)
initialiser R(i) à « no match »
Pour tout i:
	Si Sum (missing(i) .. missing (i+m) ) <=1    # Tout est missing, ou tout sauf 1 est missing
		R_i :=  « missing »
Pour tout l < l_max :    # l_max permet de ne fait qu'une partie de l'itération, utile pour débugguer
	Initialiser Q(l) :
		Soit un i(l) / R(i(l)) = « no match »    # / pour "tel que" 
			si pas possible : fin
		Q(l) := f(i(l)) … f(i(l)+m)
	Pour tout i / R(i) = « no match »  :    
		calculer d(i)
		si d(i) < Limit(m):
			R(i) := l 
Valeurs de retour : Q(l), R(i), et aussi d(i) à titre d'information.

###### just to organize my ideas,  we now have all necessary implementations:

- 'd' is a function of 'l', 'i', and 'k'. However, I think it's impractical in its current form and requires modification.
- 'Q' is a function of 'l' and utilizes vectorized versions of 'hr' and 'hg'. It contains all possible 'l' values.

- 'T' of course depends on 'k'. While it might be possible to vectorize it, but it seems it will be too much and inusual)

###### 
 'Alpha' ? |X| , I think it should be defined similarly to 'd'. (- done)

the problem is in the time taken by the code.
we can reduce the complexity by reduction the 'Q' for each step (I still need to understand the step after the test)




#

#

#

#

#

# With distance - factoor test 

In [ ]:
m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))

In [ ]:
objects = unique_ids[0:10]

L_max = int(len(objects)/2)  
print("L_max ", L_max)

d = {k: [0] * L_max for k in objects}
alp = {k: [0] * L_max for k in objects}
a = {k: [0] * L_max for k in objects}
R = {k: ['no_match'] * 1 for k in objects}
F = {k: [0] for k in objects}
W = {k: [0] for k in objects}


In [ ]:
m=0
chunk_size =2 * (m + 1)

for k in objects:

    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight', 'source']]
    n = len(T)
    F[k] = T['dc_flux'].values
    W[k] = T['dc_weight'].values
    for i in range(int(n/2)):
        box_status = (T['source'][i*chunk_size : (i+1)*chunk_size] == 0)

        if box_status.sum() <= 1:
            R[k][i*chunk_size : (i+1)*chunk_size] = ['missing']
        else:
            R[k][i*chunk_size : (i+1)*chunk_size] = ['no_match']#] *  chunk_size



In [ ]:
count = 0
# R = ['no_match']*len(objects)
Q = [None] * (L_max+1)
l= 0
while (l < L_max):
    indexes_dict = {key: [i for i, value in enumerate(values) if value == 'no_match'] for key, values in R.items()}
    
    has_non_empty_list = any(value for value in indexes_dict.values() if value)
    if not has_non_empty_list:
        break
    #print(indexes_dict)
    
    for k in R: # we can remove the loop for here ! 
        #print(l , k)
        count_match = R[k].count('no_match')
        indexes = [index for index, value in enumerate(R[k]) if value == 'no_match']
        print(indexes)

        if indexes:
            f = F[k]
#           if count_match > 0:
#             index_missing = R[k].index('no_match')
#             print(index_missing)

            index_no_match = indexes[0]
#             print(f)
            Q[l] = f[index_no_match*chunk_size : (index_no_match+1)*chunk_size]
            break
     
    
    print("HELLOOOOO",Q[l])
    
    
    for k in unique_ids[0:10]:
        f = F[k]
        w = W[k]
        n = len(f)

        zeros = np.zeros(n, dtype=float)

        s_1 = zeros.copy()
        s_2 = zeros.copy()
        d[k][l]=0
        
        for j in range(0,m+1): 
            h = np.tile(Q[l], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
                 
        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[l], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            # 2*l*(m+1) this part is to take from Q by l 
            
            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] 

        d[k][l] = dd[::2] + dd[1::2]
        
        a[k][l] =  d[k][l] <= factor 
        
        for b in indexes_dict[k]:
            if d[k][l][b] <= factor : 
                R[k][b] = l
#         test = d[k][l][indexes_dict[k]]
#         if (test <= factor).any():
#             #print(k, ": ", d[k][l],"\n")
#             count +=1
        
            
            
            
    print("print LLLLLLLLL",l)
    l += 1
        

        
    
    

#

#

#

# TEST

In [ ]:
m=0
for j in range(1, max(1, m+1)):
    print(j*2)

if m>0: 
    for j in range(1, m+1):
        print(j*2)

In [ ]:
import timeit

m = 100

code_snippet_1 = """
for j in range(1, max(1, m+1)):
    j*2
"""

code_snippet_2 = """
if m > 0:
    for j in range(1, m+1):
        j*2
"""

time_taken_1 = timeit.timeit(stmt=code_snippet_1, number=1000000, globals=globals())
time_taken_2 = timeit.timeit(stmt=code_snippet_2, number=1000000, globals=globals())

print("Time snippet 1:", time_taken_1)
print("Time snippet 2:", time_taken_2)



#

#

#

# Test for Q all possible values ! 

In [ ]:
import pandas as pd

values = list(range(10))  

# Duplicate each value
duplicated_values = [val for val in values for _ in range(2)]


df2 = pd.DataFrame({'dc_flux': duplicated_values})

print(df2)

In [ ]:
# Define the size of each chunk
m=1
chunk_size = 2 * (m + 1)

# Calculate the number of chunks
num_chunks = len(df2[['dc_flux']]) // chunk_size

# Split the DataFrame into chunks and store them in an array
Q = [df2[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

Q

In [ ]:
for j in range(1, m+1):
    Q.extend([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
    #print([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
############################### importante !!! 
Q



In [ ]:
O = pd.concat(Q)

# Display the resulting DataFrame
print(O)

In [ ]:
m=1
(len(O)) // (2*(m+1))

In [ ]:
m=2
j=2
l=0
#np.tile(Q[(j*2):(j*2+2)], (4 // 2, 1)).ravel()
O[2*l*(m+1) +(j*2) : 2*l*(m+1) +(j*2+2)]

#

#

#

#

# distance dict to dataframe 

In [ ]:
d

In [ ]:
list_of_dicts = []

# Iterate through the data dictionary and create dictionaries for each object ID
for object_id, distances in d.items():
    for distance in distances:
        list_of_dicts.append({'objectId': object_id, 'distance': distance})

# Create DataFrame from the list of dictionaries
pdf = pd.DataFrame(list_of_dicts)

# Display the DataFrame
print(pdf)

In [ ]:
len(df)/2898

In [ ]:
pdf['is_distance_gt_factor'] = pdf['distance'] <= factor

In [ ]:
pdf

# Trash

In [ ]:
df[df['objectId'] =='ZTF18acevrat' ]

In [ ]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


In [ ]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?

T = {unique_ids.index(k): df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:1]}
d = {unique_ids.index(k): np.zeros(int(len(T[k])/2), dtype=np.float64) for k in unique_ids[0:1]}

for j in range(1,m+1): 
    #h = Q.values     
    
    
            for filt in np.unique(T[k]['fid']):
                mask = T[k]['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[k][mask]['dc_flux'].values
                w = T[k][mask]['dc_sigflux_weight'].values

                s_n = (f[-1+j:]*h[j-1] * w[-1+j:])
                s_d = (h[j-1]**2 * w[-1+j:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d[k] += ((f[-1+j:] - alpha * h[j-1])**2) * w[-1+j:]

                #break
            
if (d[k] <= factor).any():
        #print(d)
        count +=1
        #print(count, k)
        
print(count)

In [ ]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d0 = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            
            alpha = 0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                s_n = (f[j:]*h[j] * w[j:])
                s_d = (h[j]**2 * w[j:])
                s = np.where(s_d == 0, 0, s_n / s_d)

                alpha += s
                #print("alpha", alpha)
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                d0[k] += ((f[j:] - alpha * h[j])**2) * w[j:]
               #print()
               #print(d)

            #break
            
    if (d0[k] <= factor).all():
        print(d0[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       